Model predicting next words part of speech

In [3]:
import spacy
from spacy.lang.en import English

In [4]:
nlp = spacy.load("en_core_web_md")

In [5]:
### Open text file as TEXT and make a doc of TEXT

with open("Hunger Games 1.txt", encoding="utf8") as f:
    TEXT = f.read()
    

In [6]:
TEXT = TEXT.replace('\n', ' ').replace('\r', '').replace('  ',' ')


In [7]:
print(TEXT[0:1000])

﻿When I wake up, the other side of the bed is cold. My fingers stretch out, seeking Prims warmth but finding only the rough canvas cover of the mattress. She must have had bad dreams and climbed in with our mother. Of course, she did. This is the day of the reaping. I prop myself up on one elbow. Theres enough light in the bedroom to see them. My little sister, Prim, curled up on her side, cocooned in my mothers body, their cheeks pressed together. In sleep, my mother looks younger, still worn but not so beaten-down. Prims face is as fresh as a raindrop, as lovely as the primrose for which she was named. My mother was very beautiful once, too. Or so they tell me. Sitting at Prims knees, guarding her, is the worlds ugliest cat. Mashed-in nose, half of one ear missing, eyes the color of rotting squash. Prim named him Buttercup, insisting that his muddy yellow coat matched the bright flower. I he hates me. Or at least distrusts me. Even though it was years ago, I think he still remembers 

In [ ]:
doc = nlp(TEXT)

In [ ]:
prev_sim = []
for i in range(1, len(doc)):
    prev_sim.append(doc[i].similarity(doc[i-1]))

In [ ]:
print(prev_sim[0:100])

In [ ]:
for i in range (0, len(doc)):
    if nlp.vocab.strings[doc[i].pos_] == 87:
        print(doc[i].text, doc[i].pos_, nlp.vocab.strings[doc[i].pos_], doc[i].dep_, nlp.vocab.strings[doc[i].dep_])

In [ ]:
### Create lists for columns the dataset:
### posb4 = part of speech of the token 4 positions before,.... posa1 = part of speech of the next word

posb4 = []
posb3 = []
posb2 = []
posb1 = []
pos0 = []
posa1 = []
depb4 = []
depb3 = []
depb2 = []
depb1 = []
dep0 = []
depa1 = []

In [ ]:
for token in doc:
    pos0.append(nlp.vocab.strings[token.pos_])
    posb4.append(nlp.vocab.strings[doc[token.i-4].pos_])
    posb3.append(nlp.vocab.strings[doc[token.i-3].pos_])
    posb2.append(nlp.vocab.strings[doc[token.i-2].pos_])
    posb1.append(nlp.vocab.strings[doc[token.i-1].pos_])
    dep0.append(nlp.vocab.strings[token.dep_])
    depb4.append(nlp.vocab.strings[doc[token.i-4].dep_])
    depb3.append(nlp.vocab.strings[doc[token.i-3].dep_])
    depb2.append(nlp.vocab.strings[doc[token.i-2].dep_])
    depb1.append(nlp.vocab.strings[doc[token.i-1].dep_])
    if token.i < len(doc) - 1:
        posa1.append(nlp.vocab.strings[doc[token.i+1].pos_])
        depa1.append(nlp.vocab.strings[doc[token.i+1].dep_])
    

In [ ]:
print(len(posb4))
print(len(posb2))
print(len(posb3))
print(len(posb1))
print(len(pos0))
print(len(posa1))
print(len(depb4))
print(len(depb2))
print(len(depb3))
print(len(depb1))
print(len(dep0))
print(len(depa1))

In [ ]:
print(len(prev_sim))

In [ ]:
print(posb4[0:10])
print(posb3[0:10])
print(posb2[0:10])
print(posb1[0:10])
print(pos0[0:10])
print(posa1[0:10])

In [ ]:
import pandas as pd

Create dataframe with columns part of speech and dependency of two previous words and similarity between them as independent variables and next word part of speech as independent variable.

In [ ]:
df = pd.DataFrame(list(zip(posb1, pos0, depb1, dep0, prev_sim, posa1)), 
               columns =["Pos i-1", "Pos i", "Dep i-1", "Dep i", "Similarity", "Pos i+1"]) 

In [ ]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=["Pos i-1", "Pos i", "Dep i-1", "Dep i"],return_df=True)
df = encoder.fit_transform(df) 

In [ ]:
# Shuffle rows
import sklearn
dataset = sklearn.utils.shuffle(df)


In [ ]:
print(dataset)

In [ ]:
print(dataset.head(10))

# Apply prediction models on dataframe df_final

In [ ]:
import numpy as np

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X[0:10])

In [ ]:
print(y[0:10])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Smaller dataset

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)

In [ ]:
classifier.fit(X_train,y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test,y_pred))

In [ ]:
### Decision trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

In [ ]:
### 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric = "minkowski", p=2)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
view = (np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

In [ ]:
print(view[0:20])

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
### SVM

In [ ]:
from sklearn import svm
# kernel default is ='rbf'
#classifier = svm.SVC()
#classifier = svm.SVC(kernel='sigmoid')
classifier = svm.SVC(kernel='linear')

In [ ]:
classifier.fit(X_train,y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)  